# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 11 2022 11:33AM,248353,19,N17794,"NAPP Technologies, LLC",Released
1,Oct 11 2022 11:27AM,248352,10,129674,"Virtue Rx, LLC",Released
2,Oct 11 2022 11:27AM,248352,10,129707,"Virtue Rx, LLC",Released
3,Oct 11 2022 11:27AM,248352,10,129723,"Virtue Rx, LLC",Released
4,Oct 11 2022 11:27AM,248352,10,129730,"Virtue Rx, LLC",Released
5,Oct 11 2022 11:27AM,248352,10,129739,"Virtue Rx, LLC",Released
6,Oct 11 2022 11:24AM,248351,19,N17795,"NAPP Technologies, LLC",Released
7,Oct 11 2022 11:17AM,248350,19,STR10112022-2,ACG North America LLC,Released
8,Oct 11 2022 11:00AM,248347,16,8624064,Sartorius Bioprocess Solutions,Released
9,Oct 11 2022 10:56AM,248346,19,8632693,VITABIOTICS USA INC.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,248347,Released,1
27,248350,Released,1
28,248351,Released,1
29,248352,Released,5
30,248353,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248347,NaN,NaN,1.0
248350,NaN,NaN,1.0
248351,NaN,NaN,1.0
248352,NaN,NaN,5.0
248353,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0.0,0.0,1.0
248264,0.0,0.0,30.0
248268,0.0,3.0,43.0
248271,0.0,1.0,0.0
248277,0.0,1.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248034,0,0,1
248264,0,0,30
248268,0,3,43
248271,0,1,0
248277,0,1,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,0,0,1
1,248264,0,0,30
2,248268,0,3,43
3,248271,0,1,0
4,248277,0,1,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248034,,,1
1,248264,,,30
2,248268,,3,43
3,248271,,1,
4,248277,,1,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC"
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC"
6,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC"
7,Oct 11 2022 11:17AM,248350,19,ACG North America LLC
8,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions
9,Oct 11 2022 10:56AM,248346,19,VITABIOTICS USA INC.
10,Oct 11 2022 10:51AM,248344,16,"SHL Pharma, LLC"
11,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.
12,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.
42,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC",,,1
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",,,5
2,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC",,,1
3,Oct 11 2022 11:17AM,248350,19,ACG North America LLC,,,1
4,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions,,,1
5,Oct 11 2022 10:56AM,248346,19,VITABIOTICS USA INC.,,,1
6,Oct 11 2022 10:51AM,248344,16,"SHL Pharma, LLC",,,1
7,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,,,1
8,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,,19,11
9,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc.",,,30


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC",1,,
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",5,,
2,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC",1,,
3,Oct 11 2022 11:17AM,248350,19,ACG North America LLC,1,,
4,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions,1,,
5,Oct 11 2022 10:56AM,248346,19,VITABIOTICS USA INC.,1,,
6,Oct 11 2022 10:51AM,248344,16,"SHL Pharma, LLC",1,,
7,Oct 11 2022 10:30AM,248343,10,Eywa Pharma Inc.,1,,
8,Oct 11 2022 10:05AM,248339,10,Eywa Pharma Inc.,11,19,
9,Oct 11 2022 9:57AM,248264,15,"Person & Covey, Inc.",30,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC",1,,
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",5,,
2,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC",1,,
3,Oct 11 2022 11:17AM,248350,19,ACG North America LLC,1,,
4,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC",1.0,NaN,NaN
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",5.0,NaN,NaN
2,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC",1.0,NaN,NaN
3,Oct 11 2022 11:17AM,248350,19,ACG North America LLC,1.0,NaN,NaN
4,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 11 2022 11:33AM,248353,19,"NAPP Technologies, LLC",1.0,0.0,0.0
1,Oct 11 2022 11:27AM,248352,10,"Virtue Rx, LLC",5.0,0.0,0.0
2,Oct 11 2022 11:24AM,248351,19,"NAPP Technologies, LLC",1.0,0.0,0.0
3,Oct 11 2022 11:17AM,248350,19,ACG North America LLC,1.0,0.0,0.0
4,Oct 11 2022 11:00AM,248347,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1490012,64.0,37.0,10.0
15,993088,112.0,4.0,0.0
16,745007,3.0,0.0,0.0
18,248317,1.0,0.0,0.0
19,993400,4.0,0.0,0.0
20,496662,25.0,32.0,25.0
21,496305,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1490012,64.0,37.0,10.0
1,15,993088,112.0,4.0,0.0
2,16,745007,3.0,0.0,0.0
3,18,248317,1.0,0.0,0.0
4,19,993400,4.0,0.0,0.0
5,20,496662,25.0,32.0,25.0
6,21,496305,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64.0,37.0,10.0
1,15,112.0,4.0,0.0
2,16,3.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,4.0,0.0,0.0
5,20,25.0,32.0,25.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,15,Released,112.0
2,16,Released,3.0
3,18,Released,1.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,25.0,0.0
Executing,37.0,4.0,0.0,0.0,0.0,32.0,1.0
Released,64.0,112.0,3.0,1.0,4.0,25.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,25.0,0.0
1,Executing,37.0,4.0,0.0,0.0,0.0,32.0,1.0
2,Released,64.0,112.0,3.0,1.0,4.0,25.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,25.0,0.0
1,Executing,37.0,4.0,0.0,0.0,0.0,32.0,1.0
2,Released,64.0,112.0,3.0,1.0,4.0,25.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()